 ### PROJETO CÁLCULO DE SCORE DE CRÉDITO DOS CLIENTES DE UM BANCO
 ##### Fomos contratados por um banco para conseguir o score de crédito dos clientes. Precisamos analisar todos os clientes do banco e, com base nessa análise, criar um modelo que consiga ler as informações do clientes e dizer automaticamente o score de crédito dele: - POOR, STANDARD ou GOOD

In [32]:
# Importando as biliotecas
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [33]:
# Importando e visualizando parte da base de dados
clientes = pd.read_csv('dados\clientes.csv')
clientes.head(10)

,id_cliente,mes,idade,profissao,salario_anual,num_contas,num_cartoes,juros_emprestimo,num_emprestimos,dias_atraso,...,idade_historico_credito,investimento_mensal,comportamento_pagamento,saldo_final_mes,score_credito,emprestimo_carro,emprestimo_casa,emprestimo_pessoal,emprestimo_credito,emprestimo_estudantil
0,3392,1,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,265.0,21.465380,alto_gasto_pagamento_baixos,312.494089,Good,1,1,1,1,0
1,3392,2,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,266.0,21.465380,baixo_gasto_pagamento_alto,284.629162,Good,1,1,1,1,0
2,3392,3,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,267.0,21.465380,baixo_gasto_pagamento_medio,331.209863,Good,1,1,1,1,0
3,3392,4,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,5.0,...,268.0,21.465380,baixo_gasto_pagamento_baixo,223.451310,Good,1,1,1,1,0
4,3392,5,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,6.0,...,269.0,21.465380,alto_gasto_pagamento_medio,341.489231,Good,1,1,1,1,0
5,3392,6,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,8.0,...,270.0,21.465380,alto_gasto_pagamento_medio,340.479212,Good,1,1,1,1,0
6,3392,7,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,271.0,21.465380,baixo_gasto_pagamento_baixo,244.565317,Good,1,1,1,1,0
7,3392,8,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,272.0,21.465380,alto_gasto_pagamento_medio,358.124168,Standard,1,1,1,1,0
8,8625,1,28.0,professor,34847.84,2.0,4.0,6.0,1.0,3.0,...,319.0,39.684018,baixo_gasto_pagamento_baixo,470.690627,Standard,0,0,0,1,0
9,8625,2,28.0,professor,34847.84,2.0,4.0,6.0,1.0,7.0,...,320.0,39.684018,alto_gasto_pagamento_alto,484.591214,Good,0,0,0,1,0


In [34]:
# Mostrando um resumo das informações

print(clientes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   id_cliente                100000 non-null  int64  
 1   mes                       100000 non-null  int64  
 2   idade                     100000 non-null  float64
 3   profissao                 100000 non-null  object 
 4   salario_anual             100000 non-null  float64
 5   num_contas                100000 non-null  float64
 6   num_cartoes               100000 non-null  float64
 7   juros_emprestimo          100000 non-null  float64
 8   num_emprestimos           100000 non-null  float64
 9   dias_atraso               100000 non-null  float64
 10  num_pagamentos_atrasados  100000 non-null  float64
 11  num_verificacoes_credito  100000 non-null  float64
 12  mix_credito               100000 non-null  object 
 13  divida_total              100000 non-null  fl

In [35]:
# Verificando os valores das variáveis categóricas
# 1 - profissao
clientes['profissao'].value_counts()

profissao
advogado         7096
engenheiro       6864
arquiteto        6824
mecanico         6776
cientista        6744
contador         6744
desenvolvedor    6720
gerente_midia    6720
professor        6672
empresario       6648
medico           6568
jornalista       6536
gerente          6432
musico           6352
escritor         6304
Name: count, dtype: int64

In [36]:
# 2 - mix_credito
clientes['mix_credito'].value_counts()

mix_credito
Normal    45848
Bom       30384
Ruim      23768
Name: count, dtype: int64

In [37]:
# 3 - comportamento_pagamento
clientes['comportamento_pagamento'].value_counts()

comportamento_pagamento
baixo_gasto_pagamento_baixo    28616
alto_gasto_pagamento_medio     19738
alto_gasto_pagamento_alto      14726
baixo_gasto_pagamento_medio    14399
alto_gasto_pagamento_baixos    11764
baixo_gasto_pagamento_alto     10757
Name: count, dtype: int64

In [38]:
# Preparando a base de dados

# Verificamos que as colunas profissao,	mix_credito, comportamento_pagamento e score_credito são object. Vamos tratar as variáveis categóricas de acordo com a sequencia abaixo:

# profissao - Variável categórica nominal - Label Encoding
# mix_credito - Variável categórica ordinal - One hot encoding
# comportamento_pagamento - Variável categórica nominal - Label Encoding


In [39]:
# Aplicando o label encoder na variavel profissao
codificador = LabelEncoder()



In [40]:
# Treina o objeto ( normalmente fazemos isso somente com dados de treino)
codificador.fit(clientes['profissao'])

LabelEncoder()

In [41]:
list(codificador.classes_)

['advogado',
 'arquiteto',
 'cientista',
 'contador',
 'desenvolvedor',
 'empresario',
 'engenheiro',
 'escritor',
 'gerente',
 'gerente_midia',
 'jornalista',
 'mecanico',
 'medico',
 'musico',
 'professor']

In [42]:
# Aplicando o objeto encoder treinado
# Faremos isso em dados de treino e teste e também em novos dados usados no modelo
clientes.profissao = codificador.transform(clientes['profissao'])

In [43]:
clientes.profissao.value_counts()

profissao
0     7096
6     6864
1     6824
11    6776
2     6744
3     6744
4     6720
9     6720
14    6672
5     6648
12    6568
10    6536
8     6432
13    6352
7     6304
Name: count, dtype: int64

In [52]:
# Criando o one hot encoding
onehot = pd.get_dummies(clientes['mix_credito'], dtype=int)

In [53]:
# Concatenanado o DataFrame onehot com o DataFrame original
clientes_novo = pd.concat([clientes, onehot], axis = 1)

In [54]:
clientes_novo.head(10)

,id_cliente,mes,idade,profissao,salario_anual,num_contas,num_cartoes,juros_emprestimo,num_emprestimos,dias_atraso,...,saldo_final_mes,score_credito,emprestimo_carro,emprestimo_casa,emprestimo_pessoal,emprestimo_credito,emprestimo_estudantil,Bom,Normal,Ruim
0,3392,1,23.0,2,19114.12,3.0,4.0,3.0,4.0,3.0,...,312.494089,Good,1,1,1,1,0,1,0,0
1,3392,2,23.0,2,19114.12,3.0,4.0,3.0,4.0,3.0,...,284.629162,Good,1,1,1,1,0,1,0,0
2,3392,3,23.0,2,19114.12,3.0,4.0,3.0,4.0,3.0,...,331.209863,Good,1,1,1,1,0,1,0,0
3,3392,4,23.0,2,19114.12,3.0,4.0,3.0,4.0,5.0,...,223.451310,Good,1,1,1,1,0,1,0,0
4,3392,5,23.0,2,19114.12,3.0,4.0,3.0,4.0,6.0,...,341.489231,Good,1,1,1,1,0,1,0,0
5,3392,6,23.0,2,19114.12,3.0,4.0,3.0,4.0,8.0,...,340.479212,Good,1,1,1,1,0,1,0,0
6,3392,7,23.0,2,19114.12,3.0,4.0,3.0,4.0,3.0,...,244.565317,Good,1,1,1,1,0,1,0,0
7,3392,8,23.0,2,19114.12,3.0,4.0,3.0,4.0,3.0,...,358.124168,Standard,1,1,1,1,0,1,0,0
8,8625,1,28.0,14,34847.84,2.0,4.0,6.0,1.0,3.0,...,470.690627,Standard,0,0,0,1,0,1,0,0
9,8625,2,28.0,14,34847.84,2.0,4.0,6.0,1.0,7.0,...,484.591214,Good,0,0,0,1,0,1,0,0


In [51]:
clientes_novo.columns

Index(['id_cliente', 'mes', 'idade', 'profissao', 'salario_anual',
       'num_contas', 'num_cartoes', 'juros_emprestimo', 'num_emprestimos',
       'dias_atraso', 'num_pagamentos_atrasados', 'num_verificacoes_credito',
       'mix_credito', 'divida_total', 'taxa_uso_credito',
       'idade_historico_credito', 'investimento_mensal',
       'comportamento_pagamento', 'saldo_final_mes', 'score_credito',
       'emprestimo_carro', 'emprestimo_casa', 'emprestimo_pessoal',
       'emprestimo_credito', 'emprestimo_estudantil', 'Bom', 'Normal', 'Ruim'],
      dtype='object')

In [58]:
# aplicando label encoding na variavel comportamento_pagamento

codificador = LabelEncoder()
codificador.fit_transform(clientes['comportamento_pagamento'])
clientes['comp_pag'] = codificador.transform(clientes['comportamento_pagamento'])

In [60]:
clientes.head(10)

,id_cliente,mes,idade,profissao,salario_anual,num_contas,num_cartoes,juros_emprestimo,num_emprestimos,dias_atraso,...,comportamento_pagamento,saldo_final_mes,score_credito,emprestimo_carro,emprestimo_casa,emprestimo_pessoal,emprestimo_credito,emprestimo_estudantil,comportamento_pagamento_encoded,comp_pag
0,3392,1,23.0,2,19114.12,3.0,4.0,3.0,4.0,3.0,...,alto_gasto_pagamento_baixos,312.494089,Good,1,1,1,1,0,1,1
1,3392,2,23.0,2,19114.12,3.0,4.0,3.0,4.0,3.0,...,baixo_gasto_pagamento_alto,284.629162,Good,1,1,1,1,0,3,3
2,3392,3,23.0,2,19114.12,3.0,4.0,3.0,4.0,3.0,...,baixo_gasto_pagamento_medio,331.209863,Good,1,1,1,1,0,5,5
3,3392,4,23.0,2,19114.12,3.0,4.0,3.0,4.0,5.0,...,baixo_gasto_pagamento_baixo,223.451310,Good,1,1,1,1,0,4,4
4,3392,5,23.0,2,19114.12,3.0,4.0,3.0,4.0,6.0,...,alto_gasto_pagamento_medio,341.489231,Good,1,1,1,1,0,2,2
5,3392,6,23.0,2,19114.12,3.0,4.0,3.0,4.0,8.0,...,alto_gasto_pagamento_medio,340.479212,Good,1,1,1,1,0,2,2
6,3392,7,23.0,2,19114.12,3.0,4.0,3.0,4.0,3.0,...,baixo_gasto_pagamento_baixo,244.565317,Good,1,1,1,1,0,4,4
7,3392,8,23.0,2,19114.12,3.0,4.0,3.0,4.0,3.0,...,alto_gasto_pagamento_medio,358.124168,Standard,1,1,1,1,0,2,2
8,8625,1,28.0,14,34847.84,2.0,4.0,6.0,1.0,3.0,...,baixo_gasto_pagamento_baixo,470.690627,Standard,0,0,0,1,0,4,4
9,8625,2,28.0,14,34847.84,2.0,4.0,6.0,1.0,7.0,...,alto_gasto_pagamento_alto,484.591214,Good,0,0,0,1,0,0,0


In [62]:
clientes = clientes.drop(['comportamento_pagamento','comportamento_pagamento_encoded'], axis=1)
clientes.head()

,id_cliente,mes,idade,profissao,salario_anual,num_contas,num_cartoes,juros_emprestimo,num_emprestimos,dias_atraso,...,idade_historico_credito,investimento_mensal,saldo_final_mes,score_credito,emprestimo_carro,emprestimo_casa,emprestimo_pessoal,emprestimo_credito,emprestimo_estudantil,comp_pag
0,3392,1,23.0,2,19114.12,3.0,4.0,3.0,4.0,3.0,...,265.0,21.46538,312.494089,Good,1,1,1,1,0,1
1,3392,2,23.0,2,19114.12,3.0,4.0,3.0,4.0,3.0,...,266.0,21.46538,284.629162,Good,1,1,1,1,0,3
2,3392,3,23.0,2,19114.12,3.0,4.0,3.0,4.0,3.0,...,267.0,21.46538,331.209863,Good,1,1,1,1,0,5
3,3392,4,23.0,2,19114.12,3.0,4.0,3.0,4.0,5.0,...,268.0,21.46538,223.451310,Good,1,1,1,1,0,4
4,3392,5,23.0,2,19114.12,3.0,4.0,3.0,4.0,6.0,...,269.0,21.46538,341.489231,Good,1,1,1,1,0,2
